# ***LEGAL DOCUMENT SUMMARIZATION - PREPROCESSING***

In [97]:
import nltk

In [98]:
#reading our dataset using pandas
import pandas as pd
judgements = pd.read_csv('/Users/meetbanthia/Documents/HaleLab/judge.csv',sep='\t',names=['filename','judgement'])
summaries = pd.read_csv('/Users/meetbanthia/Documents/HaleLab/sum.csv',sep='\t',names=['filename','summary'])

In [99]:
#Displaying all our judgements with filename with which they were stored initially
judgements

,filename,judgement
0,3644,vil Appeal No. 1948 of 1968 '. Appeal from the...
1,5235,Criminal Appeal No. 13 of 1981 From the Judgme...
2,1053,Appeal No. 222 of 1960. Appeal from the judgme...
3,1735,Appeal No. 507 of 1961. Appeal from the judgme...
4,5553,Civil Appeal No. 3947 (NT) of 1987. From the J...
...,...,...
7025,6043,ivil Appeal No. 1357 of 1973. From the Judgmen...
7026,6725,ivil Appeal No. 2123 of 1991. From the Judgmen...
7027,2543,Appeals Nos. 1153 to 1160 and 1161 to 1168 of ...
7028,3885,Civil Appeal No. 2329 of 1969. Appeal by Speci...


In [100]:
#Displaying all our summaries with filename with which they were stored initially
summaries

,filename,summary
0,3644,The appellant purchased the land in question f...
1,5235,"The appellant and the respondent, who were alr..."
2,1053,The respondents filed a suit for specific perf...
3,1735,Respondent No. 1 obtained a mortgage decree fo...
4,5553,"% The respondent, Aditya V. Birla, described t..."
...,...,...
7025,6043,One Darbar Harsurvala by virtue of a declarato...
7026,6725,The appellant was a Manager in one of the bran...
7027,2543,The assessee Electricity Board constituted und...
7028,3885,Practice and Procedure Conduct of parties to b...


In [101]:
#merging judgements and summaries dataframe into one
orig = pd.merge(judgements, summaries, on='filename')

In [102]:
orig

,filename,judgement,summary
0,3644,vil Appeal No. 1948 of 1968 '. Appeal from the...,The appellant purchased the land in question f...
1,5235,Criminal Appeal No. 13 of 1981 From the Judgme...,"The appellant and the respondent, who were alr..."
2,1053,Appeal No. 222 of 1960. Appeal from the judgme...,The respondents filed a suit for specific perf...
3,1735,Appeal No. 507 of 1961. Appeal from the judgme...,Respondent No. 1 obtained a mortgage decree fo...
4,5553,Civil Appeal No. 3947 (NT) of 1987. From the J...,"% The respondent, Aditya V. Birla, described t..."
...,...,...,...
7025,6043,ivil Appeal No. 1357 of 1973. From the Judgmen...,One Darbar Harsurvala by virtue of a declarato...
7026,6725,ivil Appeal No. 2123 of 1991. From the Judgmen...,The appellant was a Manager in one of the bran...
7027,2543,Appeals Nos. 1153 to 1160 and 1161 to 1168 of ...,The assessee Electricity Board constituted und...
7028,3885,Civil Appeal No. 2329 of 1969. Appeal by Speci...,Practice and Procedure Conduct of parties to b...


In [144]:
# 0th Judegement
orig['judgement'][1]

'Criminal Appeal No. 13 of 1981 From the Judgment and Order dated 26.3.1979 of the Allahabad High Court in Crl. Petition No. 1816 of 1976 Altar Ahmad for the Appellant (not present). V.A. Bobde (Amicus Curiae) for the Respondent. The Judgment of the Court was delivered by NATARAJAN, J. This is an unfortunate case where the High Court has quashed an order of maintenance passed in favour of a minor child Tarana Farooq by the Special Judicial Magistrate No. 1, Rampur under Section 125 Cr. P.C., in exercise of its powers under Section 482 Cr. The High Court has, however, deemed it fit to grant a certificate to the appellant Dukhtar Jahan, the mother of the minor child, under Article 134(1)(c) read with Article 134A of the Con stitution to prefer an appeal to this Court for considera tion of a question of law formulated as under: "Whether, in an application under Section 482 Cr. P.C. the High Court can interfere with concurrent findings rendered by the courts below." As we find the appeal i

In [148]:
orig['judgement'][1][13802]
Cr. P.C.

' '

In [119]:
# 0th summary
orig['summary'][0]

'The appellant purchased the land in question from Gili teppa and Nanjappa during the pendency of the land acquisi tion proceedings under the city of Bangalore Improvement Act 1945. The acquisition was for building a house colony by the Housing Board whose statutory responsibility is to implement housing schemes. The appellant challenged the acquisition before the High Court by filing a writ petition. The High Court dismissed the writ petition. In an appeal by special leave the appellant contended : 1. There was non compliance with the mandatory requirement of section 16(2) of the Act which requires the service of notice on "every person whose name appears in the Land Revenue Register as being primarily liable to pay the property tax or land revenue". Section 15(3) \'provides that the main scheme may provide for the construction of buildings for the proper and working classes including the whole or part of such classes to be displaced in the execution of this scheme. The appellant whos

Observations:


1.   There are some special characters in judegements
2.   Some punctuation marks are there
3.   Some abbreviations are there like P.H.P
4.   There are some roman numbers eg: (ii)
5.   There are some numbers like 15 in 'Section 15(3)' or some date


In [105]:
jgslist = orig["judgement"].tolist()
sumlist = orig["summary"].tolist()

In [106]:
print(type(jgslist))
print(type(sumlist))
print(type(jgslist[0]))
print(type(sumlist[0]))

<class 'list'>
<class 'list'>
<class 'str'>
<class 'str'>


In [107]:
#showing 0th judgement and its summary
print(jgslist[0])
print(sumlist[0])

vil Appeal No. 1948 of 1968 '. Appeal from the Judgment and Order dated the 21st Sept., 1967 of the Mysore High Court in W.P. No. 1168/65. S.V. Gupte with S.S. JavaIi and B. Dutta for the appellant. N. Nettar and K.R. Nagaraja, for respondents. 166 The Judgment of the Court was delivered by KRISHNA IYER, J. Two short legal issues both apparently devoid of merit were urged unsuccessfully before the High Court and repeated, with a somewhat similar fate, before us, ii we may anticipate our conclusion. A Judgment of affirma tion may usefully be an abbreviation and so, we shall brief ly deal with Shri Gupta 's twin submissions on behalf of the appellant writ petitioner. The appeal is by special leave and the subject matter is land compulsorily acquired under the City of Bangalore Improvement Act 1945 (for short the Improvement Act) (Mysore Act V of 1945). A concise narration of the necessary facts may conven iently be compressed into a paragraph or two. The appellant purchased two portions 

# Some steps that need to be done first :    

1.   Mapping of abbreviations to their full forms
2.   Legal Stopwords removal
3.   Mapping of Roman Numbers





In [169]:
# here is our abbreviations mapping dictionary
abbreviations_mappings = {
    'Act': 'Act',
    'Adv': 'Advocate',
    'A.I.R.': 'All India Reporter',
    'Anr.': 'Another',
    'Appt.': 'Appointment',
    'Art.': 'Article',
    'Assn.': 'Association',
    'Cl.': 'Clause',
    'Co.': 'Company',
    'Corp.': 'Corporation',
    'Crl.': 'Criminal',
    'D.J.': 'District Judge',
    'Edn.': 'Edition',
    'F.I.R.': 'First Information Report',
    'Govt.': 'Government',
    'H.C.': 'High Court',
    'Inc.': 'Incorporated',
    'Inst.': 'Institute',
    'J.': 'Judge',
    'Jr.': 'Junior',
    'L.J.': 'Law Journal',
    'Ltd.': 'Limited',
    'M.P.': 'Member of Parliament',
    'N.L.R.': 'National Law Review',
    'N.G.O.': 'Non-Governmental Organization',
    'No.': 'Number',
    'O.S.': 'Original Suit',
    'P.C.': 'Privy Council',
    'P.P.': 'Public Prosecutor',
    'P.W.': 'Prosecution Witness',
    'Q.C.': "Queen's Counsel",
    'R.': 'Regulation',
    'R.S.': 'Revised Statutes',
    'Rep.': 'Report',
    'S.C.': 'Supreme Court',
    'S.L.R.': 'State Law Reports',
    'S.R.': 'Section',
    'Ss.': 'Sections',
    'Sr.': 'Senior',
    'St.': 'Statute',
    'Univ.': 'University',
    'U.P.': 'Uttar Pradesh',
    'U/S': 'Under Section',
    'U.S.': 'United States',
    'V.': 'Versus',
    'Vs.': 'Versus',
    'W.P.': 'Writ Petition',
    'A.O.': 'Administrative Order',
    'B.': 'Bench',
    'B.C.C.I.': 'Board of Control for Cricket in India',
    'C.G.': 'Central Government',
    'C.J.': 'Chief Justice',
    'C.N.R.': 'Case Number',
    'Cr.': 'Criminal',
    'Cr.P.C.': 'Code of Criminal Procedure',
    'Ct.': 'Court',
    'Def.': 'Defendant',
    'Dep.': 'Department',
    'Dist.': 'District',
    'Etc.': 'Et cetera',
    'Ex.': 'Exhibit',
    'F.A.': 'First Appeal',
    'G.O.': 'Government Order',
    'G.S.R.': 'General Statutory Rules',
    'H.J.S.': 'Higher Judicial Service',
    'I.L.R.': 'Indian Law Reports',
    'Incr.': 'Increase',
    'Insp.': 'Inspector',
    'Jt.': 'Joint',
    'M.L.A.': 'Member of Legislative Assembly',
    'M.L.C.': 'Member of Legislative Council',
    'M.M.': 'Metropolitan Magistrate',
    'M.P.L.A.': 'Member of Provincial Legislative Assembly',
    'M.P.L.C.': 'Member of Provincial Legislative Council',
    'M.R.': 'Monthly Report',
    'N.I.': 'Negotiable Instrument',
    'N.O.C.': 'No Objection Certificate',
    'O.A.': 'Original Application',
    'O.P.': 'Original Petition',
    'Ord.': 'Ordinance',
    'P.': 'Page',
    'P.': 'Plaintiff',
    'P.A.': 'Power of Attorney',
    'P.M.': 'Prime Minister',
    'P.P.C.': 'Penal Code',
    'P.W.D.': 'Public Works Department',
    'R.B.I.': 'Reserve Bank of India',
    'R.C.': 'Revisional Case',
    'Reg.': 'Regulation',
    'Rev.': 'Revenue',
    'Rev.': 'Revised',
    'S.O.': 'Standing Order',
    'S.P.': 'Superintendent of Police',
    'S.S.C.': 'Secondary School Certificate',
    'S.S.L.C.': 'Secondary School Leaving Certificate',
    'S.T.': 'Scheduled Tribes',
    'S.T.': 'Sales Tax',
    'S.T.': 'Section',
    'S.T.': 'Service Tax',
    'S.W.O.': 'Service Writ Petition Order',
    'S. C. R.': 'Supreme Court Reporter',
    'Anr.': 'Another',
    'Ors.': 'Others',
    'Cr. P. C.': 'Criminal Procedure Code',
    'Smt.': 'Shrimati',
    'p.': 'Page number if followed by number',
    'Co.': 'Company',
    'Art.': 'Article',
    's.': 'Section',
    'Sec.': 'Section',
    'sub-s.': 'Sub Section',
    'viz.': 'videlicet',
    'ltd.': 'limited',
    'a.i.r.': 'All India Reporter',
    'f.i.r.': 'First Information Report',
    'i.p.c.': 'Indian Penal Code',
    'p.w.': 'Procecuted Witness',
    'cl.': 'Clauses',
    'cls.': 'Clauses',
    'ex.': 'exhibit',
    'schs.': 'schedules',
    'JJ.': 'Juvenile Justice',
    'II.': 'Indian',
    'III.': 'Third',
    'NO.': 'Number',
    'IPC.': 'Indian Penal Code',
    'CA.': 'Civil Appeal',
    'PW.': 'Prosecution Witness',
    'CJ.': 'Chief Justice',
    'IV.': 'Fourth',
    'WP.': 'Writ Petition',
    'NOS.': 'Numbers',
    'RS.': 'Rupees',
    'LL.': 'Laws',
    'SUPP.': 'Supplement',
    'VI.': 'Sixth',
    'AA.': 'Appellant Authority',
    'RR.': 'Reportable',
    'NIIL.': 'Not in Indian Law',
    'SS.': 'Sessions',
    'XIII.': 'Thirteenth',
    'RM.': 'Revisional Matter',
    'RPL.': 'Review Petition',
    'COURT.': 'Court',
    'NDMC.': 'New Delhi Municipal Council',
    'PP.': 'Public Prosecutor',
    'CPC.': 'Code of Civil Procedure',
    'US.': 'United States',
    'CSI.': 'Crime Scene Investigation',
    'EX.': 'Examination',
    'CI.': 'Criminal Investigation',
    'FF.': 'Final Findings',
    'RIL.': 'Reliance Industries Limited',
    'CO.': 'Company',
    'HELD.': 'Held',
    'VII.': 'Seventh',
    'GIPF.': 'General Insurance Pension Fund',
    'ONGC.': 'Oil and Natural Gas Corporation',
    'ST.': 'State',
    'CEGAT.': 'Customs, Excise and Gold (Control) Appellate Tribunal',
    'FFF.': 'Fully Fledged Freehold',
    'LR.': 'Land Registration',
    'FIR.': 'First Information Report',
    'HUF.': 'Hindu Undivided Family',
    'SC.': 'Supreme Court',
    'BRO.': 'Border Roads Organization',
    'XVIII.': 'Eighteenth',
    'COFEPOSA.': 'Conservation of Foreign Exchange and Prevention of Smuggling Activities Act',
    'IA.': 'Interlocutory Application',
    'IIIA.': 'Third Amendment',
    'PC.': 'Police Constable',
    'RW.': 'Review',
    'LJ.': 'Law Journal',
    'AW.': 'Appellate Watch',
    'SLP.': 'Special Leave Petition',
    'XVI.': 'Sixteenth',
    'GSI.': 'Geological Survey of India',
    'HPCL.': 'Hindustan Petroleum Corporation Limited',
    'SCR.': 'Supreme Court Reporter',
    'IX.': 'Ninth',
    'ASI.': 'Assistant Sub-Inspector',
    'DM.': 'District Magistrate',
    'VIII.': 'Eighth',
    'XI.': 'Eleventh',
    'FERA.': 'Foreign Exchange Regulation Act',
    'UBMC.': 'Unauthorised Building Monitoring Committee',
    'VA.': 'Vakalatnama',
    'BDA.': 'Bangalore Development Authority',
    'AIR.': 'All India Reporter',
    'KW.': 'Kilowatt',
    'UCC.': 'Uniform Civil Code',
    'CR.': 'Criminal',
    'SI.': 'Sub-Inspector',
    'AR.': 'Annual Report',
    'AL.': 'Advocate-on-Record',
    'CT.': 'Court',
    'XIV.': 'Fourteenth',
    'TXRS.': 'Texshare',
    'DESU.': 'Delhi Electric Supply Undertaking',
    'UCIL.': 'Union Carbide India Limited',
    'BOM.': 'Bombay',
    'LTD.': 'Limited',
    'MS.': 'Master of Science',
    'AM.': 'Advocate of Mediation',
    'DW.': 'Delhi Walla',
    'UPSC.': 'Union Public Service Commission',
    'VESCO.': 'Visakhapatnam Steel Plant Employees\' Cooperative Credit Society',
    'ORS.': 'Others',
    'CJM.': 'Chief Judicial Magistrate',
    'PL.': 'Public Law',
    'XII.': 'Twelfth',
    'UP.': 'Uttar Pradesh',
    'CRL.': 'Criminal',
    'ARC.': 'Appellate Revision Cases',
    'CMP.': 'Civil Miscellaneous Petition',
    'IT.': 'Income Tax',
    'IVA.': 'Individual Voluntary Arrangement',
    'ADSJ.': 'Additional District and Sessions Judge',
    'FACOR.': 'Ferro Alloys Corporation Limited',
    'SP.': 'Superintendent of Police',
    'SO.': 'Subordinate Officer'
}

#keys stored in decresing order of its length
sorted_keys = sorted(abbreviations_mappings.keys(), key=lambda x: len(x), reverse=True)

Question : Why so we need to store keys in decreasing order of key length?

1.  Lets say there was a word Cr. P.C. in our text
2.  AT first P.C. will be replaced and then Cr. will be replaced as they both in itself are some different abbreviation.
3. So first you will nee to check Cr. P.C. before checking for 'Cr.' and 'P.C.'
4. Thus there is a need to store keys in decreasing order

In [112]:
len(abbreviations_mappings)

213

# ERROR CODE


replacing abbreviations

    #make a copy of jdglist and sumlist and replace abbreviations listed in mappings with its value
    jgslist_new = jgslist.copy()
    sumlist_new = sumlist.copy()

    for key in abbreviations_mappings:
      #search for this key in all judgement and summaries and replace it with its mapped full form
      sz = len(jgslist_new)
      for i in range(sz):
        #replace key word with value word
        jgslist_new[i].replace(key,abbreviations_mappings[key])
        sumlist_new[i].replace(key,abbreviations_mappings[key])

ERROR:
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
/var/folders/h6/jmsygm613l765wzyqt211s100000gn/T/ipykernel_79564/3191244008.py in <module>
      6     #replace key word with value word
      7     jgslist_new[i].replace(key,abbreviations_mappings[key])
----> 8     sumlist_new[i].replace(key,abbreviations_mappings[key])

AttributeError: 'float' object has no attribute 'replace'


NOTE :   

1.  The above error says 'float' object has no attribute 'replace'
2.  Acc to this some jgslist_new[i] has type float
3.  On examining, got that 1711th summary is float
4.  We will just remove those judgements and corresponding summary if any of them has type float



In [125]:
print(len(jgslist))
print(len(sumlist))

7030
7030


In [126]:
#checking that all elements in the both the lists are strings
i = 0
while i<len(jgslist):
  if type(jgslist[i]) is not str or type(sumlist[i]) is not str:
    jgslist.pop(i)
    sumlist.pop(i)
    i -= 1
  i += 1

You can observe that there were two data whose type was not string.
As there are only two such data, we can directly remove that

In [127]:
print(len(jgslist))
print(len(sumlist))

7028
7028


In [128]:
jgslist_new = jgslist.copy()
sumlist_new = sumlist.copy()

In [129]:
print(len(jgslist_new))
print(len(sumlist_new))

7028
7028


Although we need a bigger set of mappings, this is not enough

To produce bigger set of data:
1. Web scrape the data as much as possible
2. Now use generate variations function to generate more variations of a single abbreviations.
3. That can be done by using combinations from itertools


Function to generate more variations of a single abbreviation:


    from itertools import combinations
    def generate_abbreviation_variations(abbreviation):
        n = len(abbreviation)
        variations = []
    
        # Generate variations with periods
        for r in range(2, n + 1):
            for combination in combinations(range(1, n), r):
                variation = ''.join([abbreviation[i] + '.' if i in combination else abbreviation[i] for i in range(n)])
                variations.append(variation)
    
        # Generate variations with white spaces
        for r in range(2, n + 1):
            for combination in combinations(range(1, n + 1), r):
                variation = ' '.join([abbreviation[i - 1] if i in combination else abbreviation[i - 1] + '.' for i in range(1, n + 1)])
                variations.append(variation)
    
        return variations
    
    # Example usage
    abbreviation = 'CrPC'
    
    variations = generate_abbreviation_variations(abbreviation)
    print(variations)


In [170]:
for key in sorted_keys:
  #search for this key in all judgement and summaries and replace it with its mapped full form
  sz = len(jgslist_new)
  for i in range(sz):
    #replace key word with value word
    jgslist_new[i] = jgslist_new[i].replace(key,abbreviations_mappings[key])
    sumlist_new[i] = sumlist_new[i].replace(key,abbreviations_mappings[key])

In [194]:
#both these outputs false
#means some abbreviations were replaced

print(jgslist==jgslist_new)
print(sumlist==sumlist_new)

False
False


Still there are some issues like 'Cr. P.C.'
This is still not resolved.
To fix this we have to used methods discussed before.

In [196]:
#got these legal stopwords by analysis

legal_stopwords = ['court', 'section', 'act', 'case', 'order', 'high', 'state', 'appellant', 'made', 'would', 'may', '1',
                   'government', 'appeal', 'vs', 'also', 'law', 'tax', '2', 'one', 'said', 'respondent', 'article', 'question',
                   'held', 'therefore', 'shall', 'could', 'provisions', 'must']

In [199]:
from nltk import word_tokenize
from nltk import sent_tokenize
import re
from nltk.corpus import stopwords

In [200]:
#cleaning of judgements data

#traverse through all judgements and there sumaries
for i in range(len(jgslist_new)):
  #splitting text to sentences

  #list of sentences of ith judgement
  sentences_judge = nltk.sent_tokenize(jgslist_new[i])

  #list of sentences of ith summary
  sentences_summ = nltk.sent_tokenize(sumlist_new[i])

  #Cleaning ith judgement
  #will store clean list of sentemces in corpus
  corpus = []
  for j in range(len(sentences_judge)):
    #including numbers too as dates are important
    review = re.sub('[^a-zA-Z0-9]',' ', sentences_judge[j])

    #converting each lines of the summary to lower case
    review = review.lower()

    #splitting sentence into words
    review = word_tokenize(review)

    #removing stop words
    review = [word for word in review if word not in stopwords.words('english') and word not in legal_stopwords]

    #combining words to form a sentence
    review = ' '.join(review)

    corpus.append(review)

  #removing empty strings from the list
  #observed that there were some empty strings in corpus list
  jgslist_new[i] = [x for x in corpus if x != '']

  #cleaning ith summary
  corpus = []
  for j in range(len(sentences_summ)):
    #including numbers too as dates are important
    review = re.sub('[^a-zA-Z0-9]',' ', sentences_summ[j])

    #converting each lines of the summary to lower case
    review = review.lower()

    #splitting sentence into words
    review = word_tokenize(review)

    #removing stop words
    review = [word for word in review if word not in stopwords.words('english') and word not in legal_stopwords]

    #combining words to form a sentence
    review = ' '.join(review)

    corpus.append(review)

  #removing empty strings from the list
  #observed that there were some empty strings in corpus list
  sumlist_new[i] = [x for x in corpus if x != '']

In [201]:
jgslist_new[0]

['vil number 1948 1968',
 'judgment dated 21st sept 1967 mysore writ petition number 1168 65',
 'versus gupte javaii bench dutta',
 'n nettar k regulation nagaraja respondentsection 166 judgment delivered krishna iyer judge two short legal issues apparently devoid merit urged unsuccessfully repeated somewhat similar fate us ii anticipate conclusion',
 'judgment affirma tion usefully abbreviation brief ly deal shri gupta twin submissions behalf writ petitioner',
 'special leave subject matter land compulsorily acquired city bangalore improvement 1945 short improvement mysore v 1945',
 'concise narration necessary facts conven iently compressed paragraph two',
 'purchased two portions number 211 within district bangalore two persons giliteppa nanjappa pendency land acquisition proceedings improvement',
 'proceedings acquisition land number 211 making lay plan building colony',
 'limited objective completed due formalities complied thereafter land housing board whose statutory responsibil

In [202]:
#creating copy of X_judge

#judge_k list is basically list of all judgements where all sentences in that judgement has length>K
k = 50

jgslist_latest = jgslist_new.copy()

for i in range(len(jgslist_latest)):
  jgslist_latest[i] = [sentence for sentence in jgslist_latest[i] if len(sentence)>=k]

In [206]:
len(jgslist_latest)==len(jgslist_new)

True

In [210]:
jgslist_latest[0]

['judgment dated 21st sept 1967 mysore writ petition number 1168 65',
 'n nettar k regulation nagaraja respondentsection 166 judgment delivered krishna iyer judge two short legal issues apparently devoid merit urged unsuccessfully repeated somewhat similar fate us ii anticipate conclusion',
 'judgment affirma tion usefully abbreviation brief ly deal shri gupta twin submissions behalf writ petitioner',
 'special leave subject matter land compulsorily acquired city bangalore improvement 1945 short improvement mysore v 1945',
 'concise narration necessary facts conven iently compressed paragraph two',
 'purchased two portions number 211 within district bangalore two persons giliteppa nanjappa pendency land acquisition proceedings improvement',
 'proceedings acquisition land number 211 making lay plan building colony',
 'limited objective completed due formalities complied thereafter land housing board whose statutory responsibili ty implement housing schemesection told houses built land a

# **Now save all our data in 'Pickle File'**

In [207]:
import pickle

# Save data to a pickle file
with open('/Users/meetbanthia/Documents/HaleLab/jgslist_latest.pickle', 'wb') as file:
    pickle.dump(jgslist_latest, file)

with open('/Users/meetbanthia/Documents/HaleLab/jgslist_new.pickle', 'wb') as file:
    pickle.dump(jgslist_new, file)

with open('/Users/meetbanthia/Documents/HaleLab/jgslist.pickle', 'wb') as file:
    pickle.dump(jgslist, file)

with open('/Users/meetbanthia/Documents/HaleLab/sumlist_new.pickle', 'wb') as file:
    pickle.dump(sumlist_new, file)

with open('/Users/meetbanthia/Documents/HaleLab/sumlist.pickle', 'wb') as file:
    pickle.dump(sumlist, file)

with open('/Users/meetbanthia/Documents/HaleLab/legal_stopwords.pickle', 'wb') as file:
    pickle.dump(legal_stopwords, file)

with open('/Users/meetbanthia/Documents/HaleLab/abbreviations_mappings.pickle', 'wb') as file:
    pickle.dump(abbreviations_mappings, file)